In [44]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None)

In [45]:
row_exclude = ['Total, all countries or areas', 'Africa', 'Northern Africa', 'Sub-Saharan Africa', 
              'Eastern Africa', 'Middle Africa', 'Southern Africa', 'Western Africa', 'Latin America',
              'Americas', 'Northern America', 'Latin America & the Caribbean', 'Caribbean',
              'Central America', 'South America', 'Asia', 'Central Asia', 'Eastern Asia', 
              'South-eastern Asia', 'Southern Asia', 'Western Asia', 'Europe', 'Eastern Europe',
              'Northern Europe', 'Southern Europe', 'Western Europe', 'Oceania', 'Australia and New Zealand',
              'Melanesia', 'Micronesia', 'Polynesia', 'South-central Asia', 'Caucasus', 'North America',
              'Eastern and South-Eastern Asia', 'Northern Africa and Western Asia', 'Northern America and Europe']

def read_un_data(filepath, df):   
    df = pd.read_csv(filepath, encoding = 'latin-1', skiprows = 1)
    df = df.drop(columns = ['Footnotes', 'Source', 'Region/Country/Area'])\
        .rename(columns = {'Unnamed: 1': 'Country'})
    df = df[df['Year'].isin([2005, 2010, 2015]) & ~df['Country'].isin(row_exclude)]
    return df

In [46]:
gdp = pd.DataFrame()
gdp = read_un_data('data/SYB62_T13_201904_GDP and GDP Per Capita.csv', gdp)
gdp = gdp[gdp.Series == 'GDP per capita (US dollars)']\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index().rename(columns = {'GDP per capita (US dollars)': 'gdp'})
gdp['gdp'] = gdp['gdp'].str.replace(',', '').astype(int)

gdp.head()

Series,Country,Year,gdp
0,Afghanistan,2005,264
1,Afghanistan,2010,558
2,Afghanistan,2015,611
3,Albania,2005,2615
4,Albania,2010,4056


In [47]:
pop = pd.DataFrame()
pop = read_un_data('data/SYB61_T02_Population, Surface Area and Density.csv', pop)
pop = pop[(pop.Series == 'Population mid-year estimates (millions)') | 
         (pop.Series == 'Sex ratio (males per 100 females)') | 
         (pop.Series == 'Population density')]\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index().rename(columns = {'Population mid-year estimates (millions)': 'pop',
                                    'Sex ratio (males per 100 females)': 'gender_ratio',
                                    'Population density': 'pop_density'})
pop[['pop', 'gender_ratio', 'pop_density']] = pop[['pop', 'gender_ratio', 'pop_density']]\
    .apply(lambda x: x.str.replace(',','').astype(float))

pop.head()

Series,Country,Year,pop_density,pop,gender_ratio
0,Afghanistan,2005,38.4,25.07,107.2
1,Afghanistan,2010,44.1,28.80,105.7
2,Afghanistan,2015,51.7,33.74,106.4
3,Albania,2005,112.4,3.08,100.6
4,Albania,2010,107.3,2.94,100.7


In [48]:
trade = pd.DataFrame()
trade = read_un_data('data/SYB61_T21_Total Imports, Exports and Balance of Trade.csv', trade)
trade = trade[trade.Series == 'Balance imports/exports (millions of US dollars)']\
    .drop(columns = ['System of trade', 'System of trade footnote'])\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index().rename(columns = {'Balance imports/exports (millions of US dollars)': 'exp_imp'})
trade['exp_imp'] = trade['exp_imp'].str.replace(',', '').astype(int)

trade.head()

Series,Country,Year,exp_imp
0,Afghanistan,2010,-4766
1,Afghanistan,2015,-7151
2,Albania,2005,-1956
3,Albania,2010,-3053
4,Albania,2015,-2391


In [49]:
edu = pd.DataFrame()
edu = read_un_data('data/SYB62_T07_Education.csv', edu)
edu = edu[(edu.Series == 'Students enrolled in primary education (thousands)') |
         (edu.Series == 'Students enrolled in secondary education (thousands)') | 
         (edu.Series == 'Students enrolled in tertiary education (thousands)')]\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index().rename(columns = {'Students enrolled in primary education (thousands)': 'edu_prim',
                                    'Students enrolled in secondary education (thousands)': 'edu_sec',
                                    'Students enrolled in tertiary education (thousands)': 'edu_tert'})
edu[['edu_prim', 'edu_sec', 'edu_tert']] = edu[['edu_prim', 'edu_sec', 'edu_tert']]\
    .apply(lambda x: x.str.replace(',','').astype(float))

edu.head()

Series,Country,Year,edu_prim,edu_sec,edu_tert
0,Afghanistan,2005,4318.819,651.453,NaN
1,Afghanistan,2010,5279.326,2044.157,NaN
2,Afghanistan,2015,6333.695,2698.816,NaN
3,Albania,2005,237.975,407.403,63.257
4,Albania,2010,224.781,355.871,122.326


In [50]:
gva = pd.DataFrame()
gva = read_un_data('data/SYB62_T14_Gross Value Added by Economic Activity.csv', gva)\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index()\
    .rename(columns = {'Agriculture, hunting, forestry and fishing (% of gross value added)': 'gva_ag',
                       'Industry (% of gross value added)': 'gva_indst',
                       'Services (% of gross value added)': 'gva_servc'})

gva.head()

Series,Country,Year,gva_ag,gva_indst,gva_servc
0,Afghanistan,2005,35.2479,25.9585,38.7936
1,Afghanistan,2010,28.8094,21.3476,49.8430
2,Afghanistan,2015,22.6614,22.6744,54.6642
3,Albania,2005,21.5420,28.6709,49.7872
4,Albania,2010,20.6582,28.6905,50.6513


In [51]:
labor = pd.DataFrame()
labor = read_un_data('data/SYB62_T17_201904_Labour Force and Unemployment.csv', labor)
labor = labor[(labor.Series == 'Labour force participation - Total') |
             (labor.Series == 'Unemployment rate - Total')]\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index()\
    .rename(columns = {'Labour force participation - Total': 'labor_part',
                       'Unemployment rate - Total': 'unemp'})

labor.head()

Series,Country,Year,labor_part,unemp
0,Afghanistan,2005,64.3,8.5
1,Afghanistan,2010,63.3,7.8
2,Afghanistan,2015,65.3,8.9
3,Albania,2005,57.9,17.5
4,Albania,2010,54.3,14.2


In [52]:
energy = pd.DataFrame()
energy = read_un_data('data/SYB62_T23_201904_Production, Trade and Supply of Energy.csv', energy)
energy = energy[(energy.Series == 'Primary energy production (petajoules)') |
               (energy.Series == 'Supply per capita (gigajoules)')]\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index()\
    .rename(columns = {'Primary energy production (petajoules)': 'energy_prod',
                       'Supply per capita (gigajoules)': 'energy_supp'})
energy[['energy_prod', 'energy_supp']] = energy[['energy_prod', 'energy_supp']]\
    .apply(lambda x: x.str.replace(',','').astype(float))

energy.head()

Series,Country,Year,energy_prod,energy_supp
0,Afghanistan,2005,23.0,1.0
1,Afghanistan,2010,41.0,5.0
2,Afghanistan,2015,60.0,4.0
3,Albania,2005,48.0,29.0
4,Albania,2010,69.0,31.0


In [53]:
internet = pd.DataFrame()
internet = read_un_data('data/SYB62_T29_201904_Internet Usage.csv', internet)\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index().rename(columns = {'Percentage of individuals using the internet': 'internet'})

internet.head()

Series,Country,Year,internet
0,Afghanistan,2005,1.2
1,Afghanistan,2010,4.0
2,Afghanistan,2015,8.3
3,Albania,2005,6.0
4,Albania,2010,45.0


In [54]:
tourism = pd.DataFrame()
tourism = read_un_data('data/SYB62_T30_201904_Tourist-Visitors Arrival and Expenditure.csv', tourism)
tourism = tourism[tourism.Series == 'Tourism expenditure (millions of US dollars)']\
    .drop(columns = ['Tourism arrivals series type', 'Tourism arrivals series type footnote'])\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index().rename(columns = {'Tourism expenditure (millions of US dollars)': 'tourism'})
tourism['tourism'] = tourism['tourism'].str.replace(',', '').astype(int)

tourism.head()

Series,Country,Year,tourism
0,Afghanistan,2010,147
1,Afghanistan,2015,87
2,Albania,2005,880
3,Albania,2010,1778
4,Albania,2015,1613


In [55]:
patents = pd.DataFrame()
patents = read_un_data('data/SYB61_T28_Patents.csv', patents)
patents = patents[patents.Series == 'Resident patent filings (per million population)']\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index().rename(columns = {'Resident patent filings (per million population)': 'patents'})
patents['patents'] = patents['patents'].str.replace(',', '').astype(int)

patents.head()

Series,Country,Year,patents
0,Albania,2015,5
1,Algeria,2005,2
2,Algeria,2010,2
3,Algeria,2015,2
4,Argentina,2005,27


In [56]:
health = pd.DataFrame()
health = read_un_data('data/SYB62_T11_Expenditure on Health.csv', health)
health = health[health.Series == 'Current health expenditure (% of GDP)']\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index().rename(columns = {'Current health expenditure (% of GDP)': 'health'})

health.head()

Series,Country,Year,health
0,Afghanistan,2005,9.9483
1,Afghanistan,2010,8.5697
2,Afghanistan,2015,10.0940
3,Albania,2005,6.3428
4,Albania,2010,5.0100


In [57]:
edu_GBratio = pd.DataFrame()
edu_GBratio = read_un_data('data/SYB62_T06_Ratio of Girls to Boys in Education.csv', edu_GBratio)\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index()\
    .rename(columns = {'Ratio of girls to boys in primary education': 'edu_ratio_gb_prim',
                       'Ratio of girls to boys in secondary education': 'edu_ratio_gb_sec',
                       'Ratio of girls to boys in tertiary education': 'edu_ratio_gb_tert'})

edu_GBratio.head()

Series,Country,Year,edu_ratio_gb_prim,edu_ratio_gb_sec,edu_ratio_gb_tert
0,Afghanistan,2005,0.5871,0.3252,NaN
1,Afghanistan,2010,0.6861,0.4996,NaN
2,Afghanistan,2015,0.6912,0.5637,NaN
3,Albania,2005,0.9901,0.9524,1.4246
4,Albania,2010,0.9825,0.9896,1.3380


In [58]:
parliam_women = pd.DataFrame()
parliam_women = read_un_data('data/SYB62_T05_201905_Seats held by Women in Parliament.csv', parliam_women)
parliam_women = parliam_women.drop(columns = ['Last Election Date', 'Last Election Date footnote'])\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index()\
    .rename(columns = {'Seats held by women in national parliament, as of February (%)': 'parliam_women'})

parliam_women.head()

Series,Country,Year,parliam_women
0,Afghanistan,2010,27.3
1,Afghanistan,2015,27.7
2,Albania,2005,6.4
3,Albania,2010,16.4
4,Albania,2015,20.7


In [59]:
prod_ag = pd.DataFrame()
prod_ag = read_un_data('data/SYB61_T20_Agricultural Production Indices.csv', prod_ag)
prod_ag = prod_ag[prod_ag.Series == 'Food production (Index Base: 2004-2006 = 100)']\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index().rename(columns = {'Food production (Index Base: 2004-2006 = 100)': 'food_prod'})

prod_ag.head()

Series,Country,Year,food_prod
0,Afghanistan,2005,106.6
1,Afghanistan,2010,116.2
2,Afghanistan,2015,119.6
3,Albania,2005,98.0
4,Albania,2010,119.1


In [60]:
cpi = pd.DataFrame()
cpi = read_un_data('data/SYB61_T19_Consumer Price Index.csv', cpi)
cpi = cpi[cpi.Series == 'Consumer price index: General']\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index().rename(columns = {'Consumer price index: General': 'cpi'})
cpi['cpi'] = cpi['cpi'].str.replace(',', '').astype(float)

cpi.head()

Series,Country,Year,cpi
0,Afghanistan,2005,71.1
1,Afghanistan,2010,100.0
2,Afghanistan,2015,132.1
3,Albania,2005,86.7
4,Albania,2010,100.0


In [61]:
fert = pd.DataFrame()
fert = read_un_data('data/SYB60_T03_Population Growth, Fertility and Mortality Indicators.csv', fert)
fert = fert[(fert.Series == 'Population annual rate of increase (percent)') |
           (fert.Series == 'Total fertility rate (children per women)') |
           (fert.Series == 'Infant mortality for both sexes (per 1,000 live births)') |
           (fert.Series == 'Maternal mortality ratio (deaths per 100,000 population)') |
           (fert.Series == 'Life expectancy at birth for both sexes (years)')]\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index()\
    .rename(columns = {'Population annual rate of increase (percent)': 'pop_gain',
                       'Total fertility rate (children per women)': 'num_child',
                       'Infant mortality for both sexes (per 1,000 live births)': 'mort_child', 
                       'Maternal mortality ratio (deaths per 100,000 population)': 'mort_mother', 
                       'Life expectancy at birth for both sexes (years)': 'life_exp'})
fert[['pop_gain', 'num_child', 'mort_child', 'mort_mother', 'life_exp']] = \
    fert[['pop_gain', 'num_child', 'mort_child', 'mort_mother', 'life_exp']]\
    .apply(lambda x: x.str.replace(',','').astype(float))

fert.head()

Series,Country,Year,mort_child,life_exp,mort_mother,pop_gain,num_child
0,Afghanistan,2005,89.5,56.9,821.0,4.4,7.2
1,Afghanistan,2010,76.7,60.0,584.0,2.8,6.4
2,Afghanistan,2015,68.6,62.3,396.0,3.2,5.3
3,Albania,2005,21.1,74.8,30.0,-0.3,1.9
4,Albania,2010,16.8,75.6,30.0,-0.9,1.6


In [62]:
migrants = pd.DataFrame()
migrants = read_un_data('data/SYB61_T04_International Migrants and Refugees.csv', migrants)
migrants = migrants[(migrants.Series == 'International migrant stock: Both sexes (% total population)') |
           (migrants.Series == 'Total population of concern to UNHCR (number)')]\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index()\
    .rename(columns = {'International migrant stock: Both sexes (% total population)': 'pop_migrants',
                       'Total population of concern to UNHCR (number)': 'pop_unhcr'})
migrants[['pop_migrants', 'pop_unhcr']] = migrants[['pop_migrants', 'pop_unhcr']]\
    .apply(lambda x: x.str.replace(',','').astype(float))

migrants.head()

Series,Country,Year,pop_migrants,pop_unhcr
0,Afghanistan,2005,0.3,159597.0
1,Afghanistan,2010,0.4,1199987.0
2,Afghanistan,2015,1.5,1421419.0
3,Albania,2005,2.1,92.0
4,Albania,2010,1.8,99.0


In [63]:
urban = pd.DataFrame()
urban = read_un_data('data/SYB61_T03_Population Growth Rates in Urban areas and Capital cities.csv', urban)
urban = urban[(urban.Series == 'Urban population (percent)')]\
    .drop(columns = ['Capital City', 'Capital City footnote'])\
    .pivot_table(index = ['Country', 'Year'], columns='Series', values = 'Value', aggfunc = 'max')\
    .reset_index().rename(columns = {'Urban population (percent)': 'urban'})
urban['urban'] = urban['urban'].str.replace(',', '').astype(float)

urban.head()

Series,Country,Year,urban
0,Afghanistan,2005,22.7
1,Afghanistan,2010,23.7
2,Afghanistan,2015,24.8
3,Albania,2005,46.7
4,Albania,2010,52.2


In [64]:
UN = gdp.merge(pop, on = ['Country', 'Year'], how = 'outer')\
    .merge(trade, on = ['Country', 'Year'], how = 'outer')\
    .merge(edu, on = ['Country', 'Year'], how = 'outer')\
    .merge(gva, on = ['Country', 'Year'], how = 'outer')\
    .merge(labor, on = ['Country', 'Year'], how = 'outer')\
    .merge(energy, on = ['Country', 'Year'], how = 'outer')\
    .merge(internet, on = ['Country', 'Year'], how = 'outer')\
    .merge(tourism, on = ['Country', 'Year'], how = 'outer')\
    .merge(patents, on = ['Country', 'Year'], how = 'outer')\
    .merge(health, on = ['Country', 'Year'], how = 'outer')\
    .merge(edu_GBratio, on = ['Country', 'Year'], how = 'outer')\
    .merge(parliam_women, on = ['Country', 'Year'], how = 'outer')\
    .merge(prod_ag, on = ['Country', 'Year'], how = 'outer')\
    .merge(cpi, on = ['Country', 'Year'], how = 'outer')\
    .merge(fert, on = ['Country', 'Year'], how = 'outer')\
    .merge(migrants, on = ['Country', 'Year'], how = 'outer')\
    .merge(urban, on = ['Country', 'Year'], how = 'outer')

UN.head()

Series,Country,Year,gdp,pop_density,pop,gender_ratio,exp_imp,edu_prim,edu_sec,edu_tert,gva_ag,gva_indst,gva_servc,labor_part,unemp,energy_prod,energy_supp,internet,tourism,patents,health,edu_ratio_gb_prim,edu_ratio_gb_sec,edu_ratio_gb_tert,parliam_women,food_prod,cpi,mort_child,life_exp,mort_mother,pop_gain,num_child,pop_migrants,pop_unhcr,urban
0,Afghanistan,2005,264.0,38.4,25.07,107.2,NaN,4318.819,651.453,NaN,35.2479,25.9585,38.7936,64.3,8.5,23.0,1.0,1.2,NaN,NaN,9.9483,0.5871,0.3252,NaN,NaN,106.6,71.1,89.5,56.9,821.0,4.4,7.2,0.3,159597.0,22.7
1,Afghanistan,2010,558.0,44.1,28.80,105.7,-4766.0,5279.326,2044.157,NaN,28.8094,21.3476,49.8430,63.3,7.8,41.0,5.0,4.0,147.0,NaN,8.5697,0.6861,0.4996,NaN,27.3,116.2,100.0,76.7,60.0,584.0,2.8,6.4,0.4,1199987.0,23.7
2,Afghanistan,2015,611.0,51.7,33.74,106.4,-7151.0,6333.695,2698.816,NaN,22.6614,22.6744,54.6642,65.3,8.9,60.0,4.0,8.3,87.0,NaN,10.0940,0.6912,0.5637,NaN,27.7,119.6,132.1,68.6,62.3,396.0,3.2,5.3,1.5,1421419.0,24.8
3,Albania,2005,2615.0,112.4,3.08,100.6,-1956.0,237.975,407.403,63.257,21.5420,28.6709,49.7872,57.9,17.5,48.0,29.0,6.0,880.0,NaN,6.3428,0.9901,0.9524,1.4246,6.4,98.0,86.7,21.1,74.8,30.0,-0.3,1.9,2.1,92.0,46.7
4,Albania,2010,4056.0,107.3,2.94,100.7,-3053.0,224.781,355.871,122.326,20.6582,28.6905,50.6513,54.3,14.2,69.0,31.0,45.0,1778.0,NaN,5.0100,0.9825,0.9896,1.3380,16.4,119.1,100.0,16.8,75.6,30.0,-0.9,1.6,1.8,99.0,52.2


In [65]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_greenhouse_gas_emissions_per_capita'
response = requests.get(url)
response.status_code

200

In [66]:
page = response.text
soup = BeautifulSoup(page, 'html5')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of countries by greenhouse gas emissions per capita - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_countries_by_greenhouse_gas_emissions_per_capita","wgTitle":"List of countries by greenhouse gas emissions per capita","wgCurRevisionId":905962987,"wgRevisionId":905962987,"wgArticleId":12976556,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia articles in need of updating from June 2019","All Wikipedia articles in need of updating","All articles with unsourced statements","Articles with unsourced statements from February 2012","Greenhouse gas emissions","Lists of countries","Lists of countr

In [67]:
table = soup.find('table', {'class': 'wikitable sortable'})
print(table)

<table class="wikitable sortable">
<caption>Metric Tons of <a href="/wiki/Carbon_dioxide_equivalent" title="Carbon dioxide equivalent">CO<sub>2</sub>e</a> Per capita
</caption>
<tbody><tr>
<th>Country
</th>
<th>1990
</th>
<th>1995
</th>
<th>2000
</th>
<th>2005
</th>
<th>2010
</th>
<th>2013
</th></tr>
<tr>
<td><span class="flagicon"><img alt="" class="thumbborder" data-file-height="600" data-file-width="1200" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Flag_of_Kuwait.svg/23px-Flag_of_Kuwait.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Flag_of_Kuwait.svg/35px-Flag_of_Kuwait.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Flag_of_Kuwait.svg/46px-Flag_of_Kuwait.svg.png 2x" width="23"/> </span><a href="/wiki/Kuwait" title="Kuwait">Kuwait</a>
</td>
<td>37.32
</td>
<td>73.13
</td>
<td>65.64
</td>
<td>76.41
</td>
<td>62.47
</td>
<td>54.41
</td></tr>
<tr>
<td><span class="flagicon"><img alt="" class="thumbborder

In [68]:
links = table.find_all('a')
countries = []
for country in links:
    countries.append(country.get('title'))
ghg = pd.DataFrame()
ghg['Country'] = countries[1:]
ghg.head()

,Country
0,Kuwait
1,Brunei
2,Niue
3,Qatar
4,Belize


In [69]:
rows = table.find_all('tr')
rows = [row for row in rows if ('World' not in row.text) and ('Country' not in row.text)]
year_2005 = []
year_2010 = []
year_2013 = []

for i in range(0, len(rows)):
    columns = rows[i].find_all('td')
    year_2005.append(columns[4].get_text().strip())
    year_2010.append(columns[5].get_text().strip())
    year_2013.append(columns[6].get_text().strip())

year_2005 = [np.nan if x == '' else x for x in year_2005]
year_2010 = [np.nan if x == '' else x for x in year_2010]
year_2013 = [np.nan if x == '' else x for x in year_2013]

ghg['2005'] = year_2005
ghg['2010'] = year_2010
ghg['2015'] = year_2013
ghg.head()

,Country,2005,2010,2015
0,Kuwait,76.41,62.47,54.41
1,Brunei,44.83,46.88,46.84
2,Niue,24.3,28.65,45.6
3,Qatar,52.1,39.13,36.82
4,Belize,28.11,28.29,28.15


In [70]:
ghg = ghg.melt(id_vars = ['Country'], value_vars = ['2005', '2010', '2015'], 
               var_name = 'Year', value_name = 'ghg_emissions')
ghg[['Year', 'ghg_emissions']] = ghg[['Year', 'ghg_emissions']].transform({'Year': int, 'ghg_emissions': float})
ghg.head(10)

,Country,Year,ghg_emissions
0,Kuwait,2005,76.41
1,Brunei,2005,44.83
2,Niue,2005,24.30
3,Qatar,2005,52.10
4,Belize,2005,28.11
5,Oman,2005,25.84
6,Bahrain,2005,27.78
7,Australia,2005,27.60
8,United Arab Emirates,2005,32.13
9,Libya,2005,20.91


In [71]:
ghg.Country[~ghg.Country.isin(UN.Country)].unique()

array(['Brunei', 'United States', 'Russia', 'South Korea',
       'Republic of Ireland', 'Czech Republic', 'Iran', 'The Bahamas',
       'Venezuela', 'Bolivia', 'The Gambia', 'Georgia (country)', 'Syria',
       'Moldova', 'Vietnam', 'North Korea',
       'Saint Vincent and the Grenadines', 'Laos', 'Tanzania',
       'Republic of the Congo', 'Ivory Coast', 'Cape Verde',
       'São Tomé and Príncipe', 'Democratic Republic of the Congo'],
      dtype=object)

In [72]:
ghg = ghg.replace({
    'Republic of Ireland': 'Ireland',
    'The Bahamas': 'Bahamas',
    'The Gambia': 'Gambia',
    'Georgia (country)': 'Georgia',
    'Saint Vincent and the Grenadines': 'Saint Vincent & Grenadines',
    'Republic of the Congo': 'Congo',
    'São Tomé and Príncipe': 'Sao Tome and Principe',
    'Democratic Republic of the Congo': 'Dem. Rep. of the Congo'
})

UN = UN.replace({
    'Brunei Darussalam': 'Brunei',
    'United States of America': 'United States',
    'Russian Federation': 'Russia',
    'Republic of Korea': 'South Korea',
    'Czechia': 'Czech Republic',
    'Iran (Islamic Republic of)': 'Iran',
    'Venezuela (Boliv. Rep. of)': 'Venezuela',
    'Bolivia (Plurin. State of)': 'Bolivia',
    'Syrian Arab Republic': 'Syria',
    'Republic of Moldova': 'Moldova',
    'Viet Nam': 'Vietnam',
    "Dem. People's Rep. Korea": 'North Korea',
    "Lao People's Dem. Rep.": 'Laos',
    'United Rep. of Tanzania': 'Tanzania',
    'Côte d\x92Ivoire': 'Ivory Coast',
    'Cabo Verde': 'Cape Verde'    
})

In [73]:
UN = UN.merge(ghg, on = ['Country', 'Year'], how = 'left')
UN.head(10)

,Country,Year,gdp,pop_density,pop,gender_ratio,exp_imp,edu_prim,edu_sec,edu_tert,gva_ag,gva_indst,gva_servc,labor_part,unemp,energy_prod,energy_supp,internet,tourism,patents,health,edu_ratio_gb_prim,edu_ratio_gb_sec,edu_ratio_gb_tert,parliam_women,food_prod,cpi,mort_child,life_exp,mort_mother,pop_gain,num_child,pop_migrants,pop_unhcr,urban,ghg_emissions
0,Afghanistan,2005,264.0,38.4,25.07,107.2,NaN,4318.819,651.453,NaN,35.2479,25.9585,38.7936,64.3,8.5,23.0,1.0,1.2,NaN,NaN,9.9483,0.5871,0.3252,NaN,NaN,106.6,71.1,89.5,56.9,821.0,4.4,7.2,0.3,159597.0,22.7,0.76
1,Afghanistan,2010,558.0,44.1,28.80,105.7,-4766.0,5279.326,2044.157,NaN,28.8094,21.3476,49.8430,63.3,7.8,41.0,5.0,4.0,147.0,NaN,8.5697,0.6861,0.4996,NaN,27.3,116.2,100.0,76.7,60.0,584.0,2.8,6.4,0.4,1199987.0,23.7,1.06
2,Afghanistan,2015,611.0,51.7,33.74,106.4,-7151.0,6333.695,2698.816,NaN,22.6614,22.6744,54.6642,65.3,8.9,60.0,4.0,8.3,87.0,NaN,10.0940,0.6912,0.5637,NaN,27.7,119.6,132.1,68.6,62.3,396.0,3.2,5.3,1.5,1421419.0,24.8,1.40
3,Albania,2005,2615.0,112.4,3.08,100.6,-1956.0,237.975,407.403,63.257,21.5420,28.6709,49.7872,57.9,17.5,48.0,29.0,6.0,880.0,NaN,6.3428,0.9901,0.9524,1.4246,6.4,98.0,86.7,21.1,74.8,30.0,-0.3,1.9,2.1,92.0,46.7,2.68
4,Albania,2010,4056.0,107.3,2.94,100.7,-3053.0,224.781,355.871,122.326,20.6582,28.6905,50.6513,54.3,14.2,69.0,31.0,45.0,1778.0,NaN,5.0100,0.9825,0.9896,1.3380,16.4,119.1,100.0,16.8,75.6,30.0,-0.9,1.6,1.8,99.0,52.2,2.83
5,Albania,2015,3895.0,106.7,2.92,102.0,-2391.0,188.371,315.079,160.527,22.5215,24.7798,52.6987,56.5,17.1,88.0,31.0,63.3,1613.0,5.0,6.8197,0.9731,0.9269,1.5305,20.7,139.3,111.4,14.6,77.7,29.0,-0.1,1.7,1.8,8098.0,57.4,2.85
6,Algeria,2005,3100.0,14.0,33.29,102.3,25645.0,4361.744,3654.232,792.121,8.0130,59.7320,32.2550,42.5,15.3,7534.0,48.0,5.8,477.0,2.0,3.2352,0.9244,1.0953,1.2787,6.2,98.7,81.8,37.3,71.5,148.0,1.3,2.4,0.6,94407.0,63.8,4.04
7,Algeria,2010,4463.0,15.2,36.12,101.9,16051.0,3312.440,4616.226,1144.271,8.6209,51.4168,39.9622,42.4,10.0,6200.0,45.0,12.5,324.0,2.0,5.1172,0.9369,1.0354,1.4436,7.7,130.3,100.0,32.6,73.9,147.0,1.6,2.7,0.6,94448.0,67.5,4.58
8,Algeria,2015,4163.0,16.7,39.87,102.0,-17007.0,3925.429,NaN,1289.474,12.0836,37.2842,50.6322,41.5,11.2,5883.0,56.0,38.2,347.0,2.0,6.9829,0.9515,NaN,1.5635,31.6,151.1,126.7,27.7,75.3,140.0,2.0,3.0,0.6,100036.0,70.8,4.88
9,Andorra,2005,41281.0,167.8,0.08,109.0,-1653.0,4.085,3.737,0.342,0.4433,17.3054,82.2513,NaN,NaN,0.0,130.0,37.6,NaN,NaN,8.5660,NaN,NaN,NaN,14.3,NaN,89.6,NaN,NaN,NaN,3.7,1.2,63.8,NaN,90.3,NaN


In [74]:
UN.to_pickle('UN_data.pickle')